In this notebook, we show more examples of how `HierarchicalUtils.jl` can be used for simplifying operations with tree-structured data

### BSON visualization

Let's say we want to see under the hood of `BSON` file representation. We can do this as follows

In [1]:
using Pkg
pkg"activate"
using BSON
using HierarchicalUtils

 Activating environment at `~/.julia/dev/HierarchicalUtils/examples/Project.toml`


In [2]:
import HierarchicalUtils: NodeType, noderepr, children, InnerNode, LeafNode

NodeType(::Type{T}) where T <: Union{AbstractString, Number, Symbol} = LeafNode()
NodeType(::Type{DataType}) = LeafNode()
NodeType(::Type{T}) where T <: Union{Dict, Vector} = InnerNode()

children(n::Dict{T, <:Any}) where T <: Symbol =(; n...)
children(n::Dict{T, <:Any}) where T <: AbstractString =(; Dict(Symbol(k)=>v for (k, v) in n)...)
children(n::Vector) = tuple(n...)

noderepr(n::T) where T <: Union{AbstractString, Number, DataType, Symbol} ="$n"
noderepr(n::Dict) = "Dict of"
noderepr(n::Vector) = "Vector of"

noderepr (generic function with 4 methods)

Note that the same behavior can be achieved for `Vector`s and `Dicts` with macros `@hierarchical_dict` and `@hierarchical_vector`, respectively. In a similar vain, `@hierarchical_tuple`, `@hierarchical_namedtuple` and `@hierarchical_pairvector` are available. Also, `@primitives` macro defines all primitive types and strings in julia as leaves.

Let's create some dummy data to test it with

In [3]:
a = [:Foo, "Bar", [1, "Baz", [π], r".*"]]
buff = IOBuffer()
BSON.bson(buff, Dict(:a => a))
seek(buff, 0)
bson_struct = BSON.parse(buff)

Dict{Symbol,Any} with 1 entry:
  :a => Any[Dict{Symbol,Any}(:tag=>"symbol",:name=>"Foo"), "Bar", Any[1, "Baz",…

In [4]:
printtree(bson_struct)

Dict of
  └── a: Vector of
           ├── Dict of
           │     ├── name: Foo
           │     └─── tag: symbol
           ├── Bar
           └── Vector of
                 ├── 1
                 ├── Baz
                 ├── Dict of
                 │     ├── data: Vector of
                 │     ├── size: Vector of
                 │     │           └── 1
                 │     ├─── tag: array
                 │     └── type: Dict of
                 │                 ├──── name: Vector of
                 │                 │             ├── Main
                 │                 │             ├── Base
                 │                 │             └── Irrational
                 │                 ├── params: Vector of
                 │                 │             └── Dict of
                 │                 │                   ├── name: π
                 │                 │                   └─── tag: symbol
                 │                 └───── tag: datatype
       

In [5]:
printtree(bson_struct; trav=true)

Dict of [""]
  └── a: Vector of ["U"]
           ├── Dict of ["c"]
           │     ├── name: Foo ["e"]
           │     └─── tag: symbol ["g"]
           ├── Bar ["k"]
           └── Vector of ["s"]
                 ├── 1 ["t"]
                 ├── Baz ["u"]
                 ├── Dict of ["v"]
                 │     ├── data: Vector of ["v6"]
                 │     ├── size: Vector of ["vE"]
                 │     │           └── 1 ["vI"]
                 │     ├─── tag: array ["vM"]
                 │     └── type: Dict of ["vU"]
                 │                 ├──── name: Vector of ["vW"]
                 │                 │             ├── Main ["vWU"]
                 │                 │             ├── Base ["vX*"]
                 │                 │             └── Irrational ["vXU"]
                 │                 ├── params: Vector of ["vY"]
                 │                 │             └── Dict of ["vZ"]
                 │                 │                   ├── name

In [6]:
walk(bson_struct, "wNU")

Dict{Symbol,Any} with 3 entries:
  :tag    => "datatype"
  :params => Any[Dict{Symbol,Any}(:tag=>"datatype",:params=>Any[],:name=>Any["C…
  :name   => Any["Core", "Ptr"]